# Password Game RL Training with VERL

Train Qwen3-0.6B to solve the Password Game using PPO.

**Rules**: 9 progressive password rules  
**Reward**: +1 per rule passed, -0.1 per character

In [ ]:
!nvidia-smi -L

In [ ]:
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q flash-attn --no-build-isolation
!pip install -q transformers accelerate datasets tokenizers wandb tqdm numpy

In [ ]:
!uv pip install google-genai openai

In [ ]:
import os, json, random, time, re
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
import wandb
from tqdm.auto import tqdm

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "GPU required"

In [ ]:
import os
from google import genai
from google.genai import types

## Config

# Gemini for eval

In [ ]:
@dataclass
class Config:
    # Model
    model_name: str = "Qwen/Qwen3-0.6B"
    precision: str = "bfloat16"
    use_flash_attn: bool = True
    
    # Training
    num_epochs: int = 3
    num_steps_per_epoch: int = 100
    batch_size: int = 4
    samples_per_prompt: int = 4
    learning_rate: float = 1e-6
    weight_decay: float = 0.01
    max_grad_norm: float = 1.0
    warmup_steps: int = 50
    
    # PPO
    ppo_epochs: int = 4
    clip_range: float = 0.2
    value_loss_coef: float = 0.1
    kl_coef: float = 0.05
    gamma: float = 0.99
    gae_lambda: float = 0.95
    normalize_advantages: bool = True
    
    # Generation
    max_prompt_length: int = 1024
    max_new_tokens: int = 256
    temperature: float = 0.8
    top_p: float = 0.9
    top_k: int = 50
    
    # Password Game
    num_rules: int = 9
    reward_per_rule: float = 1.0
    length_penalty: float = 0.1
    
    # Data
    num_train_samples: int = 1000
    num_val_samples: int = 200
    
    # Logging
    wandb_project: str = "password-game-rl"
    wandb_run_name: Optional[str] = None
    log_interval: int = 10
    eval_interval: int = 50
    save_interval: int = 100
    output_dir: str = f"./password_game_{int(time.time())}"
    seed: int = 42
    
    def __post_init__(self):
        if self.wandb_run_name is None:
            self.wandb_run_name = f"password_ppo_{int(time.time())}"
        os.makedirs(self.output_dir, exist_ok=True)

config = Config()
print(f"Model: {config.model_name}")
print(f"Batch: {config.batch_size} x {config.samples_per_prompt} = {config.batch_size * config.samples_per_prompt}")
print(f"Output: {config.output_dir}")

with open(os.path.join(config.output_dir, "config.json"), "w") as f:
    json.dump(asdict(config), f, indent=2)

In [ ]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(config.seed)

## Models

In [ ]:
GEMINI_API_KEY = "" #add gemini api key
client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
dtype = torch.bfloat16 if config.precision == "bfloat16" else torch.float32

tokenizer = AutoTokenizer.from_pretrained(config.model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
print(f"Tokenizer: {len(tokenizer)} tokens")

In [ ]:
from IPython.display import Markdown


system_instruction = """
  You are an expert at creating password based on the given rules. You will get the instructions of the game at teh start, follow those and play the game till you can. 
"""

chat_config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    tools=[{"google_search": {}}],
    thinking_config=types.ThinkingConfig(
      include_thoughts=True
    )
)

chat = client.chats.create(
    model="gemini-2.5-pro",
    config=chat_config,
)

In [ ]:
policy_model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2" if config.use_flash_attn else "eager"
)
policy_model.config.use_cache = False
print(f"Policy: {sum(p.numel() for p in policy_model.parameters())/1e9:.2f}B params")

In [ ]:
reference_model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2" if config.use_flash_attn else "eager"
)
reference_model.eval()
for param in reference_model.parameters():
    param.requires_grad = False
print("Reference: frozen")

# Openai utils

In [ ]:
class ValueHead(nn.Module):
    def __init__(self, hidden_size: int):
        super().__init__()
        self.linear = nn.Linear(hidden_size, 1)
        nn.init.orthogonal_(self.linear.weight, gain=0.01)
        nn.init.constant_(self.linear.bias, 0.0)
    
    def forward(self, hidden_states):
        return self.linear(hidden_states)

value_head = ValueHead(policy_model.config.hidden_size).to(DEVICE).to(dtype)
print(f"Value head: {policy_model.config.hidden_size}")

In [ ]:
OPENAI_API_KEY = ""
import os
import json
from openai import OpenAI


client = OpenAI(api_key=OPENAI_API_KEY)

SCHEMA = {
    "type": "json_schema",
    "strict":True,
    "name": "AnswerAndDate",
        "schema": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "answer": {
                    "type": "string",
                    "description": "The concise answer or summary."
                },
                "date": {
                    "type": "string",
                    "format": "date",
                    "description": "Today's date in YYYY-MM-DD."
                }
            },
            "required": ["answer", "date"]
        }
}

def ask_with_search(prompt: str,  model: str = "gpt-4.1", temperature: float = 0.1) -> dict:
    """
    Generic function that calls OpenAI Responses API with web search enabled.

    Args:
        prompt: The search query/prompt
        schema: The JSON schema for response format
        model: OpenAI model to use
        temperature: Temperature for response generation

    Returns:
        dict: Parsed JSON response based on provided schema
    """
    response = client.responses.create(
        model=model,
        input=prompt,
        text={
            "format": SCHEMA
        },
        tools=[
            {
                "type": "web_search",
                "user_location": {
                    "type": "approximate"
                },
                "search_context_size": "medium"
            }
        ],
        temperature=temperature,
        top_p=1,
        store=True,
        include=["web_search_call.action.sources"]
    )
    return json.loads(response.output_text)

def utils_get_wordle() -> dict:
    """
    Gets today's Wordle answer using web search.
    Returns: { "answer": str, "date": "YYYY-MM-DD" }
    """
    prompt = "What is today's wordle answer? In your answer only include the word, no other text"
    return ask_with_search(prompt,temperature=1)

def utils_get_emoji() -> dict:
    """
    Gets today's actual moon phase emoji using web search.
    Returns: { "emoji": str, "date": "YYYY-MM-DD" }
    """
    prompt = "What is today's current moon phase? Return only the appropriate emoji: 🌑 🌒 🌓 🌔 🌕 🌖 🌗 🌘"
    return ask_with_search(prompt, temperature=0.1)

## Password Game Environment

# game class

In [ ]:
PASSWORD_RULES = [
    "Your password must be at least 5 characters.",
    "Your password must include a number.",
    "Your password must include an uppercase letter.",
    "Your password must include a special character.",
    "The digits in your password must add up to 25.",
    "Your password must include a month of the year.",
    "Your password must include a roman numeral.",
    "Your password must include one of our sponsors: (Pepsi, Starbucks, Shell)",
    "The roman numerals in your password should multiply to 35.",
]

INSTRUCTIONS = """You are playing a password game. Create a password that satisfies ALL the given rules.
Return ONLY the password string, nothing else."""

def check_rule(password: str, rule_idx: int) -> bool:
    pwd = password
    
    if rule_idx == 0:
        return len(pwd) >= 5
    elif rule_idx == 1:
        return any(c.isdigit() for c in pwd)
    elif rule_idx == 2:
        return any(c.isupper() for c in pwd)
    elif rule_idx == 3:
        return any(not c.isalnum() for c in pwd)
    elif rule_idx == 4:
        return sum(int(c) for c in pwd if c.isdigit()) == 25
    elif rule_idx == 5:
        months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
        return any(m in pwd.lower() for m in months)
    elif rule_idx == 6:
        return bool(re.search(r'[IVXLCDM]+', pwd))
    elif rule_idx == 7:
        sponsors = ['pepsi', 'starbucks', 'shell']
        return any(s in pwd.lower() for s in sponsors)
    elif rule_idx == 8:
        romans = re.findall(r'[IVXLCDM]+', pwd)
        if not romans:
            return False
        roman_vals = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
        product = 1
        for r in romans:
            val = 0
            prev = 0
            for c in reversed(r):
                v = roman_vals.get(c, 0)
                if v < prev:
                    val -= v
                else:
                    val += v
                prev = v
            if val > 0:
                product *= val
        return product == 35
    return False

def compute_reward(password: str, num_active_rules: int) -> float:
    passing = sum(check_rule(password, i) for i in range(num_active_rules))
    rule_score = passing * config.reward_per_rule
    length_penalty = len(password) * config.length_penalty
    return rule_score - length_penalty

def format_prompt(num_active_rules: int) -> str:
    rules_text = "\n".join([f"{i+1}. {PASSWORD_RULES[i]}" for i in range(num_active_rules)])
    return f"{INSTRUCTIONS}\n\nRules:\n{rules_text}\n\nPassword:"

print(f"Loaded {len(PASSWORD_RULES)} rules")
print(f"Example prompt:\n{format_prompt(3)[:200]}...")

In [ ]:
import random
import string
import re
import requests
from typing import List, Dict, Optional, Set
from datetime import datetime
import json

def create_captcha():
    """Generates a random five-character combination of 3 letters and 2 numbers."""
    letters = [random.choice(string.ascii_letters) for _ in range(3)]
    numbers = [random.choice(string.digits) for _ in range(2)]
    captcha_list = letters + numbers
    random.shuffle(captcha_list)
    captcha = ''.join(captcha_list)
    return captcha.lower()

countries = [
    "Albania", "Algeria", "Andorra", "Armenia", "Austria", "Bahrain", "Belarus",
    "Belgium", "Bolivia", "Burundi", "Comoros", "Croatia", "Czechia", "Denmark",
    "Ecuador", "Eritrea", "Estonia", "Finland", "Georgia", "Germany", "Grenada",
    "Hungary", "Iceland", "Ireland", "Jamaica", "Lebanon", "Lesotho", "Liberia",
    "Moldova", "Morocco", "Myanmar", "Namibia", "Nigeria", "Romania", "Senegal",
    "Somalia", "Tunisia", "Ukraine", "Uruguay", "Vanuatu", "Vietnam"
]

def random_country():
    """Gives options of random seven letter countries for the game."""
    return random.choice(countries)

def get_wordle_answer():
    """Fetch today's Wordle answer from web."""
    try:
        response = utils_get_wordle()
        if response and "answer" in response:
            return response["answer"].lower()
    except:
        pass
    return "crane"  # Fallback answer

def get_current_moon_phase():
    """Get current moon phase emoji using OpenAI + search."""
    try:
        #result = utils_get_emoji()
        #return result["answer"]
        return "🌕"
    except:
        raise Exception("Cannot determine real moon phase - API unavailable")

rules = [
    "Your password must be at least 5 characters.",
    "Your password must include a number.",
    "Your password must include an uppercase letter.",
    "Your password must include a special character.",
    "The digits in your password must add up to 25.",
    "Your password must include a month of the year.",
    "Your password must include a roman numeral.",
    "Your password must include one of our sponsors: (Pepsi, Starbucks, Shell) - case insensitive",
    "The roman numerals in your password should multiply to 35.",
    "Your password must include this CAPTCHA: {captcha}",
    "Your password must include today's Wordle answer.",
    "Your password must include a two letter symbol from the periodic table.",
    "Your password must include the current phase of the moon as an emoji",
    "Your password must include the name of this country: {country}",
    "Your password must include a leap year.",
    "🥚 ← This is my chicken Paul. He hasn't hatched yet, please put him in your password and keep him safe.",
    "The elements in your password must have atomic numbers that add up to 200.",
    "Your password is not strong enough 🏋️‍♂️",
    "Your password must contain one of the following affirmations: (I am loved, I am worthy, I am enough)",
    "Paul has hatched! Please don't forget to feed him, he eats three 🐛.",
    "A sacrifice must be made. Pick two letters that you will no longer be able to use.",
    "Your password must include this color in hex. green",
    "Your password must include the length of your password.",
    "The length of your password must be a prime number.",
    "Uh lets skip this one",
    "Your password must contain three consecutive letters that appear in alphabetical order (like 'abc', 'hij', 'xyz')"
]

instructions = """
You are playing a password game. After each step, you will be given a rule and the password has to comply to that rule and all the previous rules.
Keep thinking and only submit as final answer if you are confident it's correct.
If you can't get it, ask for a hint.
You are allowed to use search.
Goal is minimum length of final password.

GOAL: When given a rule, in your answer, only give me the password string you have that satisfies the rule.

Some clarifications as you may need them:
1. Only capital letters count towards Roman Numbers - if it's a valid roman number.
2. For a substring to count as an element, first (or only) letter of the symbol has to be capitalized.
3. For other answers, you can play around with capitalization or not to satisfy the given rules. Checks are case insensitive othersie - not for roman or elements.
4. If you have a conflict, state that and say you want to end the game.

RESPONSE GUIDELINES:
respond in markdown in following format: 
### Password
{password}
### Giveup
{True if you want to give up, false otherwise}
"""

class PasswordGame:
    def __init__(self):
        self.current_rule = 0
        self.game_active = True
        self.captcha = create_captcha()
        self.country = random_country()
        self.wordle_answer = get_wordle_answer()
        self.moon_phase = get_current_moon_phase()
        self.password_history = []


    def get_current_rule(self) -> Optional[str]:
        if self.current_rule >= len(rules) or not self.game_active:
            return None
        rule = rules[self.current_rule]
        if "{captcha}" in rule:
            return rule.format(captcha=self.captcha)
        elif "{country}" in rule:
            return rule.format(country=self.country)
        return rule

    def get_all_rules_up_to_current(self) -> List[str]:
        formatted_rules = []
        for i, rule in enumerate(rules[:self.current_rule + 1]):
            if "{captcha}" in rule:
                formatted_rules.append(rule.format(captcha=self.captcha))
            elif "{country}" in rule:
                formatted_rules.append(rule.format(country=self.country))
            else:
                formatted_rules.append(rule)
        return formatted_rules

    def advance_rule(self):
        self.current_rule += 1
        if self.current_rule >= len(rules):
            self.game_active = False

    def end_game(self):
        self.game_active = False

    def calculate_reward(self, password: str) -> float:
        """Calculate reward: +1 per passing rule, -0.1 per character."""
        satisfied_rules = 0

        # Check all rules up to current rule (inclusive when game ends)
        rule_count = self.current_rule if self.game_active else len(rules)

        for i in range(rule_count):
            if self._check_rule(password, i):
                satisfied_rules += 1

        # +1 per passing rule, -0.1 per character
        rule_score = satisfied_rules
        length_penalty = len(password) * 0.1
        total_reward = rule_score - length_penalty

        return round(total_reward, 1)

    def get_rule_feedback(self, password: str) -> Dict:
        """Get detailed feedback on which rules pass/fail."""
        feedback = {
            "password": password,
            "length": len(password),
            "rules_checked": [],
            "total_passing": 0,
            "reward": 0.0
        }

        # For feedback, include current rule if game is active
        rule_count = (self.current_rule + 1) if self.game_active else len(rules)

        for i in range(rule_count):
            passes = self._check_rule(password, i)
            rule_text = rules[i]
            if "{captcha}" in rule_text:
                rule_text = rule_text.format(captcha=self.captcha)
            elif "{country}" in rule_text:
                rule_text = rule_text.format(country=self.country)

            feedback["rules_checked"].append({
                "rule_index": i,
                "rule_text": rule_text,
                "passes": passes
            })
            if passes:
                feedback["total_passing"] += 1

        feedback["reward"] = self.calculate_reward(password)
        return feedback

    def _check_rule(self, password: str, rule_index: int) -> bool:
        """Comprehensive rule checking for all password rules."""
        if rule_index == 0:  # At least 5 characters
            return len(password) >= 5

        elif rule_index == 1:  # Include a number
            return any(c.isdigit() for c in password)

        elif rule_index == 2:  # Include uppercase letter
            return any(c.isupper() for c in password)

        elif rule_index == 3:  # Include special character
            return any(not c.isalnum() for c in password)

        elif rule_index == 4:  # Digits sum to 25
            digit_sum = sum(int(c) for c in password if c.isdigit())
            return digit_sum == 25

        elif rule_index == 5:  # Include month
            months = ['january', 'february', 'march', 'april', 'may', 'june',
                     'july', 'august', 'september', 'october', 'november', 'december']
            return any(month in password.lower() for month in months)

        elif rule_index == 6:  # Include roman numeral
            roman_pattern = r'[IVXLCDM]+'
            return bool(re.search(roman_pattern, password))

        elif rule_index == 7:  # Include sponsor
            sponsors = ['pepsi', 'starbucks', 'shell']
            return any(sponsor in password.lower() for sponsor in sponsors)

        elif rule_index == 8:  # Roman numerals multiply to 35
            return self._check_roman_multiply(password, 35)

        elif rule_index == 9:  # Include CAPTCHA
            return self.captcha in password.lower()

        elif rule_index == 10:  # Include Wordle answer
            return self.wordle_answer.lower() in password.lower()

        elif rule_index == 11:  # Include periodic element
            return self._check_periodic_element(password)

        elif rule_index == 12:  # Include moon phase emoji
            return self.moon_phase in password

        elif rule_index == 13:  # Include country (dynamic)
            return self.country.lower() in password.lower()

        elif rule_index == 14:  # Include leap year
            return self._check_leap_year(password)

        elif rule_index == 15:  # Include Paul (egg emoji)
            return '🥚' in password

        elif rule_index == 16:  # Atomic numbers sum to 200
            return self._check_atomic_sum(password, 200)

        elif rule_index == 17:  # Not strong enough (always fails)
            return False

        elif rule_index == 18:  # Include affirmation
            affirmations = ['i am loved', 'i am worthy', 'i am enough']
            return any(affirmation in password.lower() for affirmation in affirmations)

        elif rule_index == 19:  # Paul eats 3 bugs
            return password.count('🐛') == 3

        elif rule_index == 20:  # Sacrifice two letters
            # This would need state tracking - simplified for now
            return True

        elif rule_index == 21:  # Include green hex
            green_hex_pattern = r'#00[89ab]000|#008000'
            return bool(re.search(green_hex_pattern, password.lower()))

        elif rule_index == 22:  # Include password length
            return str(len(password)) in password

        elif rule_index == 23:  # Length is prime
            return self._is_prime(len(password))

        elif rule_index == 24:  # Skip this one
            return True

        elif rule_index == 25:  # Include 3 consecutive chars    
            # Normalize to lowercase for case-insensitive checking
            pwd = password.lower()
    
            for i in range(len(pwd) - 2):
                triplet = pwd[i:i+3]
        
                # Must be three letters
                if triplet.isalpha():
                    # Check if they're consecutive: a->b->c, etc.
                    if ord(triplet[2]) - ord(triplet[0]) == 2:
                        # Also ensure no gaps: b must be between a and c
                        if ord(triplet[1]) == ord(triplet[0]) + 1:
                            return True
            return False

        return False


    def _check_roman_multiply(self, password: str, target: int) -> bool:
        """Check if roman numerals multiply to target."""
        romans = re.findall(r'[IVXLCDM]+', password)
        if not romans:
            return False

        roman_vals = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
        product = 1

        for r in romans:
            val = 0
            prev = 0
            for c in reversed(r):
                v = roman_vals.get(c, 0)
                if v < prev:
                    val -= v
                else:
                    val += v
                prev = v
            if val > 0:
                product *= val

        return product == target

    def _check_periodic_element(self, password: str) -> bool:
        """Check if password contains a periodic element (two-letter symbol)."""
        periodic_elements = [
            "He", "Li", "Be", "Ne", "Na", "Mg", "Al", "Si", "Cl", "Ar", "Ca",
            "Sc", "Ti", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge",
            "As", "Se", "Br", "Kr", "Rb", "Sr", "Zr", "Nb", "Mo", "Tc", "Ru",
            "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "Xe", "Cs", "Ba",
            "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho",
            "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "Re", "Os", "Ir", "Pt", "Au",
            "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
            "Pa", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No",
            "Lr", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn", "Nh",
            "Fl", "Mc", "Lv", "Ts", "Og"
        ]
        return any(elem in password for elem in periodic_elements)

    def _check_leap_year(self, password: str) -> bool:
        """Check if password contains a leap year (4-digit year divisible by 4)."""
        years = re.findall(r'\b\d{4}\b', password)
        for year in years:
            y = int(year)
            if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0):
                return True
        return False

    def _check_atomic_sum(self, password: str, target: int) -> bool:
        """Check if atomic numbers of elements sum to target."""
        element_atomic = {
            "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8,
            "F": 9, "Ne": 10, "Na": 11, "Mg": 12, "Al": 13, "Si": 14, "P": 15,
            "S": 16, "Cl": 17, "Ar": 18, "K": 19, "Ca": 20, "Sc": 21, "Ti": 22,
            "V": 23, "Cr": 24, "Mn": 25, "Fe": 26, "Co": 27, "Ni": 28, "Cu": 29,
            "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36,
            "Rb": 37, "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43,
            "Ru": 44, "Rh": 45, "Pd": 46, "Ag": 47, "Cd": 48, "In": 49, "Sn": 50,
            "Sb": 51, "Te": 52, "I": 53, "Xe": 54, "Cs": 55, "Ba": 56, "La": 57,
            "Ce": 58, "Pr": 59, "Nd": 60, "Pm": 61, "Sm": 62, "Eu": 63, "Gd": 64,
            "Tb": 65, "Dy": 66, "Ho": 67, "Er": 68, "Tm": 69, "Yb": 70, "Lu": 71,
            "Hf": 72, "Ta": 73, "W": 74, "Re": 75, "Os": 76, "Ir": 77, "Pt": 78,
            "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, "Bi": 83, "Po": 84, "At": 85,
            "Rn": 86, "Fr": 87, "Ra": 88, "Ac": 89, "Th": 90, "Pa": 91, "U": 92,
            "Np": 93, "Pu": 94, "Am": 95, "Cm": 96, "Bk": 97, "Cf": 98, "Es": 99,
            "Fm": 100, "Md": 101, "No": 102, "Lr": 103, "Rf": 104, "Db": 105,
            "Sg": 106, "Bh": 107, "Hs": 108, "Mt": 109, "Ds": 110, "Rg": 111,
            "Cn": 112, "Nh": 113, "Fl": 114, "Mc": 115, "Lv": 116, "Ts": 117, "Og": 118
        }

        found_elements = []
        for elem in element_atomic.keys():
            if elem in password:
                found_elements.append(elem)

        total = sum(element_atomic[e] for e in found_elements)
        return total == target

    def _is_prime(self, n: int) -> bool:
        """Check if a number is prime."""
        if n < 2:
            return False
        if n == 2:
            return True
        if n % 2 == 0:
            return False
        for i in range(3, int(n**0.5) + 1, 2):
            if n % i == 0:
                return False
        return True

    def play_turn(self, password: str) -> Dict:
        """Main game loop function."""
        if not self.game_active:
            return {
                "status": "game_over",
                "message": "Game has ended",
                "reward": self.calculate_reward(password)
            }

        self.password_history.append(password)
        feedback = self.get_rule_feedback(password)

        # Check if current password satisfies all active rules
        all_rules_satisfied = all(
            rule["passes"] for rule in feedback["rules_checked"]
        )

        if all_rules_satisfied:
            self.advance_rule()
            if not self.game_active:
                return {
                    "status": "won",
                    "message": "Congratulations! You've passed all rules!",
                    "feedback": feedback,
                    "reward": self.calculate_reward(password)
                }
            else:
                return {
                    "status": "success",
                    "message": f"Passed! Now try rule {self.current_rule + 1}",
                    "next_rule": self.get_current_rule(),
                    "feedback": feedback,
                    "reward": self.calculate_reward(password)
                }
        else:
            return {
                "status": "failed",
                "message": "Password doesn't satisfy all rules",
                "current_rule": self.get_current_rule(),
                "feedback": feedback,
                "reward": self.calculate_reward(password)
            }

In [ ]:
class PasswordDataset(Dataset):
    def __init__(self, num_samples: int, max_rules: int):
        self.prompts = []
        self.num_rules = []
        for _ in range(num_samples):
            n = random.randint(3, min(max_rules, len(PASSWORD_RULES)))
            self.prompts.append(format_prompt(n))
            self.num_rules.append(n)
    
    def __len__(self):
        return len(self.prompts)
    
    def __getitem__(self, idx):
        return {'prompt': self.prompts[idx], 'num_rules': self.num_rules[idx]}

train_dataset = PasswordDataset(config.num_train_samples, 9)
val_dataset = PasswordDataset(config.num_val_samples, 9)
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")